# Lab 3: Exploratory Data Analysis and Unsupervised Learning

## Introduction

Exploratory Data Analysis (EDA) is an important step in the Data Science process. EDA involves the investigation of a dataset using various graphical and statistical techniques in order to identify underlying relationships, characteristics, and patterns within the data.

<br/>

When performing EDA, we aim to answer questions such as:
   * how many observations and features are present in the dataset?
   * is the data labelled or unlabelled?
   * does the dataset consist of time-series, sequence, or independent data points?
   * how is the feature data distributed?
   * are the features numerical, categorical, or both?
   * is there evidence of correlation between features?
   * is the dataset balanced?
   * will you need to use encoding to convert categorical features to numerical features?
   * does the dataset have missing or null values?
   * how will missing or null values be handled?
   * are there obvious outliers in the data?
   * will dimensionality reduction be required for the dataset?
   * should the data be normalized?

<br/>

Although this lab should not be considered an exhaustive review of EDA, the following exercises will provide you with the opportunity to work through several example EDA techniques, answering many of the above questions in the process.

<br/>
<br/>
<br/>

## Exercise 1 - Exploratory Data Analysis

First, we need to define our imports - the set of libraries that will help us to explore our dataset.

For this notebook, our imports can be organized into 3 categories:
   * **numerical calculation and data analysis:** pandas, numpy
   * **visualization:** matplotlib, seaborn
   * **machine learning:** sklearn

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

<br/>

Begin by loading a test dataset - in this case, [Fisher's iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), a famous multivariate dataset originally published in 1936 by Ronald Fisher.

For convenience, this dataset is included with the scikit-learn library, and is a popular dataset for exploring machine learning concepts such as clustering and classification.

The iris dataset can be loaded using the *load_iris()* helper method:


In [ ]:
iris = datasets.load_iris()

<br/>

---

**Note:** When loading a custom dataset, you obviously will not have access to such a helper method.

For future reference, custom datsets can easily be loaded directly into a pandas DataFrame using a command such as:
> df = pd.read_csv("/path/to/your/file.csv")

See: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

We will cover *pandas* and *DataFrames* later in this lab.

---

<br/>
<br/>

Invoking a variable will often provide useful information. Let's try this for our *iris* variable:

In [ ]:
iris

<br/>

By inspecting the output, we see that the *iris* variable references a dictionary-like data structure that contains several useful elements:

| Element | Purpose |
|---|:---|
| DESCR | contains a plain-text description of the dataset |
| data | contains an array of data, with rows representing individual observations, and columns representing features |
| feature_names | contains a list of our feature names |
| filename | shows the local file system path of the dataset data file |
| target | contains an array of the class IDs for each of our observations |
| target_names | contains the target class labels |

<br/>

In this lab, we will make use of *data*, *target*, *feature_names*, and *target_names*.

<br/>

Let's take a few moments to explore the *data* element of *iris*. This can be achieved by calling `iris.data` or `iris['data']`:

In [ ]:
 iris['data']

<br/>

Next, let's determine the dimensions of the *iris.data* array by calling `iris.data.shape`:


In [ ]:
iris.data.shape

<br/>

We can see that the *iris.data* array has *150* rows and *4* columns.

<br/>

Next, let's examine the list of feature names via `iris.feature_names`:

In [ ]:
iris.feature_names

<br/>

For the iris dataset, the 4 features consist of measurements of sepal and petal widths & lengths.

<br/>

Next, let's take a look at the `iris.target` array:

In [ ]:
iris.target

<br/>

The *iris.target* array maps each of the observations from our *iris.data* array to one of three target IDs: 0, 1, or 2.

<br/>

---

**Note:** For today's exercise, it is very helpful for us to use a dataset that contains the target ID (known label) for each observation. In practice, you will not always have the luxury of working with a dataset with known labels.

---

<br/>

The numerical target IDs can be mapped to their corresponding text equivalent via the `iris.target_names` array:

In [ ]:
iris.target_names

In the above output:
   * ID 0 represents 'setosa'
   * ID 1 represents 'versicolor'
   * ID 2 represents 'virginica'
   
These are the 3 species of iris found in our dataset.

<br/>

Before we begin analyzing the dataset, we will first load the data into a pandas DataFrame.

<br/>

---

If you are not familiar with pandas, it is "an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language." (https://pandas.pydata.org/)

The pandas libray is widely used for data exploration and analysis.

pandas has 2 main types of data structures:
   * **Series** a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.)
   * **DataFrame** a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a dict of Series objects. It is generally the most commonly used pandas object.

We will use both types of pandas data structures in today's exercises.

---

<br/>



Let's begin by creating a new DataFrame called *df*:


In [ ]:
df = pd.DataFrame(iris.data)

<br/>

You can examine your newly created DataFrame by calling `df`:

In [ ]:
df

<br/>

You will notice that the pandas DataFrame provides an easier to read representation of the data, as compared to the *iris.data* array output that we examined in an earlier step.

<br/>

You can also use the DataFrame *head()* helper method to show the first 5 rows of data in the DataFrame:

In [ ]:
df.head()

<br/>

As it stands, the column names in our DataFrame are not particularly useful. Let's replace the numerical IDs with the feature names from our dataset, and re-run `df.head()`:

In [ ]:
df.columns = iris.feature_names
df.head()

<br/>

Compared to the previous output, this is much better!

<br/>

Next, let's use some additional DataFrame helper methods to see if there are any *null* values in our dataset:

In [ ]:
df.isnull().sum()

<br/>

Similarly, let's see if there are any *missing* values in our dataset:

In [ ]:
df.isna().sum()

<br/>

Examining the above output, you will see that the iris dataset does not have any missing/null values. This is convenient for today's exercise, but perhaps not the norm.

<br/>

---

**Note:** In practice,  whenever you encounter null/missing values in a dataset, you will need to make a decision on how to handle those values.

For example, you might choose to:
   * drop the affected rows or columns
   * impute the missing values (ex: replace each missing value with the mean value within the given feature)
   * interpolate the missing values based on adjacent values in the dataset
   * leverage a downstream machine learning algorithm that is robust to missing values
   
For additional details, see https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html

---

<br/>

Moving on, let's add the corresponding numerical target IDs (representing species) to the DataFrame:

In [ ]:
df['species'] = pd.Series(iris.target)
df.head()

Examining the above output, you will notice a new column called 'species' has been added to our DataFrame, consisting of a pandas Series that was created using the values in the *iris.target* array.

<br/>

To make the *species* column easier to understand, we will replace the numerical target IDs in the *species* column with the actual species names:

In [ ]:
df['species'] = df.species.map(pd.Series(iris.target_names))
df.head()

Great! Things are starting to come together!

<br/>

Now that we have all of the iris data in a DataFrame, let's explore some additional DataFrame helper methods.

First, let's try `df.info()`:

In [ ]:
df.info()

The DataFrame *info()* method provides a concise overview of the contents of the DataFrame.

In the output above, we can quickly see that the iris dataset consists of 150 observations each containing 4 *numerical* features, plus the *species* categorical feature that we just added.

<br/>

---

**Note:** Many machine learning algorithms (including the ones we will use today) require numerical features as input. If your dataset contains non-numerical (categorical) features that we want to include in our analysis, we would often choose to encode the categorical features into numerical features using a process called *feature engineering*.

Examples include [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) and [label encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder).

For an explanation of when to use *one-hot encoding* versus *label encoding*, refer to the following [article](https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621).

---

<br/>

We will be exluding the *species* column from our machine learning analyses, so we will not need to perfrom *feature engineering* today.

<br/>

Next, let's try `df.describe()`:

In [ ]:
df.describe()

<br/>

The DataFrame *describe()* method is a quick & easy way to capture basic statistics for all numerical columns in a DataFrame.

<br/>

With DataFrames, you can easily perform aggregations. Let's say you are interested in the quantity of observations associated with each species in our dataset. This can be accomplished using the *groupby()* method:

In [ ]:
df.groupby('species').size()

<br/>

You will notice that in our case, each of the 3 species classes contain the same number of observations (50 each). We consider this dataset balanced (an ideal situation).

<br/>

---

**Note:** In practice, you will encounter datasets for which the classes are not balanced - a situation that can have implications for machine learning algorithms. Please refer to [this article](https://medium.com/james-blogs/handling-imbalanced-data-in-classification-problems-7de598c1059f) to explore methods for dealing with unbalanced datasets.

---

<br/>

Next, let's further explore the dataset...

In many cases, insights can be gained by investigating the distribution of feature data within a dataset. A commonly used tool for visualizing distributions is the [histogram](https://en.wikipedia.org/wiki/Histogram).

Conveniently, the matplotlib library provides an easy to use *hist()* method to generate a histogram for a given DataFrame.

Let's create a histogram for the 'sepal length' feature by calling `df['sepal length (cm)'].hist(bins=40)`:

In [ ]:
df['sepal length (cm)'].hist(bins=20)

Easy, right?

Repeat the same procedure for the other 3 features, and compare the distributions. Feel free to adjust the value of the *bins* parameter, and observe how it changes the histograms.

<br/>

With a few extra lines of code, we can easily visualize and compare the distributions of all 4 features (sepal length, sepal width, petal length, petal width) at the same time:

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(12,8))
fig.suptitle("Distribution of sepal/petal lengths and widths", fontsize=18)
h1 = axs[0].hist(df['sepal length (cm)'], bins=20)
h2 = axs[1].hist(df['sepal width (cm)'], bins=20, color='r')
h3 = axs[2].hist(df['petal length (cm)'], bins=20, color='g')
h4 = axs[3].hist(df['petal width (cm)'], bins=20, color='c')
fig.legend(loc='center right', fontsize=13)
plt.subplots_adjust(right=0.78, top=0.93)

<br/>

In the above histograms, you will see that sepal length and sepal width appear to be [normally distributed](https://en.wikipedia.org/wiki/Normal_distribution), whereas petal length and petal width exhibit [bimodal distributions](https://en.wikipedia.org/wiki/Multimodal_distribution).

<br/>

The [density plot](https://en.wikipedia.org/wiki/Kernel_density_estimation) is a similar method for visualizing data distributions.

Let's create a combined density plot for all 4 features using the *kdeplot()* method from the seaborn library:


In [ ]:
plt.figure(figsize=(14,4))
sns.kdeplot(df['sepal length (cm)'], shade=True, bw='scott')
sns.kdeplot(df['sepal width (cm)'], shade=True, bw='scott')
sns.kdeplot(df['petal length (cm)'], shade=True, bw='scott')
sns.kdeplot(df['petal width (cm)'], shade=True, bw='scott')

<br/>

Again, note that *sepal length* and *sepal width* appear to be normally distributed, while *petal length* and *petal width* show bimodal distributions.

<br/>

In the source code for the above density plot, the bandwidth *(bw)* value was estimated using Scott's Rule *(D.W. Scott, “Multivariate Density Estimation: Theory, Practice, and Visualization”, John Wiley & Sons, New York, Chicester, 1992.)*.

The choice of bandwidth value will affect the overall smoothing of the density plot. It is sometimes useful to choose a custom value for *bw* in order to further investigate the underlying distribution.

Let's change the value of *bw* and see what happens:

In [ ]:
plt.figure(figsize=(14,4))
sns.kdeplot(df['sepal length (cm)'], shade=True, bw=0.1)
sns.kdeplot(df['sepal width (cm)'], shade=True, bw=0.1)
sns.kdeplot(df['petal length (cm)'], shade=True, bw=0.1)
sns.kdeplot(df['petal width (cm)'], shade=True, bw=0.1)

Notice the difference in smoothing between this density plot and the one generated in the previous step.

Try re-generating the above density plots using several different values of *bw*. Note how *bw* affects the appearance of the density plots.

<br/>
<br/>

When you are exploring a dataset with multiple features, it is often interesting to plot the features against one another to see if there are any noticeable relationships or structures.

The seaborn library has a helpful method called *pairplot()* that conveniently handles all of the heavy lifting for us:


In [ ]:
sns.pairplot(data=df,palette="Set2")

<br/>
In the above plot, the upper-left to lower-right diagonal tiles depict histograms for the 4 features in the iris dataset. The surrounding tiles show scatterplots for all pairwise combinations of the 4 features in our dataset.

Notice that in all of the scatterplots, you can clearly see 2 clusters present.

In practice, you will often find yourself exploring "unlabelled" data that does not have any target labels associated with the individual observations (much like we see in the above pairplot). Looking at the above pairplot, we can not determine which data points are associated with which class in our dataset. We can see the presence of 2 clusters, but the class assignment of the data points is unknown.

For this exercise, we actually do have labelled data. Let's repeat the *pairplot()*, this time coloring each observation based on its known target label (species):

In [ ]:
sns.pairplot(data=df,hue='species',palette="Set1")
plt.show()

By examining the above scatterplots, you can clearly observe that that the data points for a given species tend to fall within the same general region in the scatterplots. You will also see that the the data points are organized into 2 reasonably well defined clusters. Data points associated with *setosa* are easily separable from *versicolor* and *virginica*, whereas *versicolor* and *virginica* overlap to some degree.

<br/>

---

**Note:** It is worth mentioning that the iris dataset has a relatively small number of features, and can easily be visualized in a pairplot. For datasets with more than a handful of features, you will often benefit from leveraging some form of dimensionality reduction, such as [Principal Component Analysis](https://en.wikipedia.org/wiki/Dimensionality_reduction#Principal_component_analysis_(PCA)).

---

<br/>

Let's finish off this section with 2 more examples of methods for visualizing data distributions: 1) the boxplot, and 2) the violin plot:

In [ ]:
plt.figure(figsize=(14,6))
sns.boxplot(data=df);

In [ ]:
plt.figure(figsize=(12,8))
sns.violinplot(data=df, orient='h', palette="Set3");

Compare these plots to your previous histograms and density plots. Do you notice any outliers?

<br/>
<br/>
<br/>
<br/>
<br/>

# Exercise 2 - Unsupervised Learning (K-means Clustering)

<br/>

In this section, we will apply unsupervised learning to the iris dataset. Specifically, we will apply the [k-means clustering](https://en.wikipedia.org/wiki/K-means_clustering) algorithm to the dataset to attempt to organize the data points into a set number of clusters.

<br/>

---

**Note:** Please keep in mind that unsupervised learning is usually applied to unlabelled datasets. In our case, we know that the iris dataset contains target labels. For the purpose of this analysis, however, we will exclude the labels (species) from the clustering process, simulating the analysis of an unlabelled dataset.

So for today - let's pretend that the iris dataset contains unlabelled data.

---

<br/>

Begin by creating a new DataFrame *(df2)* from our original DataFrame, and removing the *species* column:

In [ ]:
df2 = df.drop(columns = 'species')
df2.head()

Notice that we have now removed our target labels *(species)*, and now have an unlabelled dataset.

<br/>

Often times, you will need to normalize your dataset before applying a given machine learning algorithm. In general, the k-means algorithm will usually benefit from normalization, so we will perform the normalization process here.

Let's use the *StandardScaler* method from the sklearn library to normalize our data:

In [ ]:
scaler = StandardScaler()
df2[df2.columns] = scaler.fit_transform(df2[df2.columns])
df2.head()

Notice that the dataset values have been scaled from their original values.

<br/>
<br/>

Before we apply the k-means algorithm to our dataset, we need to choose an appropriate value for _k_, which represents the number of clusters that will be produced by the k-means algorithm. In our case, we already know that there are 3 species of iris in the dataset, and so '3' would obviously be a great choice for *k*.

In practice, though, you will not usually know the number of clusters in advance.

<br/>

So - how should you choose an appropriate value of *k* when the number of classes is not known in advance?

<br/>

Luckily there are several methods for choosing a reasonably good value for *k*.

Below, we will use the [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), which begins by applying k-means to the dataset over a range of values of _k_ and recording the [*inertia*](https://scikit-learn.org/stable/modules/clustering.html) for each value:

In [ ]:
# source: https://blog.cambridgespark.com/how-to-determine-the-optimal-number-of-clusters-
# for-k-means-clustering-14f27070048f

Sum_of_squared_distances = []

K = range(1,15)

for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df2)
    Sum_of_squared_distances.append(km.inertia_)

<br/>

Next, we will plot the various *inertia* values, in terms of their corresponding *k* values:

In [ ]:
# source: https://blog.cambridgespark.com/how-to-determine-the-optimal-number-of-clusters-
# for-k-means-clustering-14f27070048f

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

With the elbow method, the idea is to choose the _k_ value at which the *inertia* or *sum of squared distances* begins to level off.

In the above graph, you can see that the curve levels off around *k* == 3.

We will therefore apply the k-means algorithm with a *k* value of 3:

In [ ]:
km_cluster = KMeans(n_clusters=3)
km = km_cluster.fit(df2)

<br/>

Now that we have applied the *k-means* algorithm, let's take a look at the cluster assignment that was produced by the algorithm.

The cluster assignment (labels) are available via `km.labels_`:

In [ ]:
km.labels_

Above, you see that the *k-means* algorithm has provided us with an array of 150 labels, corresponding to the 150 observations in our unlabelled dataset stored in DataFrame *df2*.

<br/>

Let's take a few moments to add the cluster labels to our *df2* DataFrame.

First, let's display the first 5 rows of our DataFrame (for comparison):

In [ ]:
df2.head()

Next, we add a new 'cluster' column to the DataFrame, using the values from *km.labels_*.

To make it easier to understand the 'cluster' column, we map the numerical values (0, 1, 2) to textual values (cluster1, cluster2, cluster3):

In [ ]:
df2['cluster'] = pd.Series(km.labels_)
df2['cluster'] = df2.cluster.map(pd.Series(['cluster1','cluster2','cluster3']))
df2.head()

With our cluster labels in place, let's do a *pairplot()*, coloring the data points based on their cluster assignment:

In [ ]:
sns.pairplot(data=df2,hue='cluster',palette="Set1")
plt.show()

You will notice a striking similarity to the *pairplot()* plots of our original (species-labelled) dataset, generated earlier on in this lab.

From visual inspection, the *k-means* algorithm has done a fair job at organizing the unlabelled data points into 3 clusters, which appear to represent our 3 iris species.

<br/>

Let's take a closer look, and plot *petal width* versus *petal length* for both our original labelled dataset (our ground truth), and the new clustered dataset:

In [ ]:
sns.lmplot(x="petal width (cm)", y="petal length (cm)", data=df, fit_reg=False, hue='species', legend=True)
sns.lmplot(x="petal width (cm)", y="petal length (cm)", data=df2, fit_reg=False, hue='cluster', legend=True, \
           palette="Paired")

Inspecting the above plots, you will see that the *k-means* results are good, but not perfect. The *setosa* data points have been clustered with high accuracy, but there are obviously data points for *versicolor* and *virginica* that have been assigned to the incorrect cluster.

Considering that *k-means* is an unsupervised approach, these results are still quite promising.

<br/>
<br/>

By comparing our known *species* labels with our *k-means* cluster labels in a frequency table, we can determine which cluster represents which species:

In [ ]:
ct1 = pd.crosstab(df['species'],df2['cluster'])
sns.heatmap(ct1,annot=True,cbar=False,cmap="Blues")

In the table above, you will see that one cluster represents all 50 of the *setosa* data points, one cluster represents mostly *versicolor* data points, and the other cluster represents mostly *virginica* data points.

<br/>

Based on the above results, even if the iris dataset did not contain the target/species labels, we could use the *k-means algorithm* to organize the unlabelled data points into 3 clusters, roughly representing the 3 iris species.

<br/>

Consider how this method could be useful when you are approaching unlabelled datasets.

<br/>
<br/>
<br/>

# Exercise 3 - Unsupervised Learning (Hierarchical Clustering)

In this final section, you will repeat the unsupervised learning process, this time by applying a [hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering) algorithm to unlabelled iris data.

<br/>
<br/>

Begin by creating a new DataFrame *(df3)* from DataFrame *(df2)*, and dropping the *cluster* column:

In [ ]:
df3 = df2.drop(columns='cluster')
df3.head()

You will see that we are starting with an unlabelled dataset - much like you will encounter while analyzing real-world datasets in practice.

<br/>
<br/>

Similar to the the *k-means* algorithm, our *hierarchical clustering* algorithm also requires a parameter that indicates that number of clusters that will be generated from our data.

---

As we have already performed the *elbow method* in a previous step, we will reuse the discovered value (3) here rather than repeating the process.

---

Let's apply the hierarchical clustering algorithm to our DataFrame:

In [ ]:
hc_cluster = AgglomerativeClustering(n_clusters=3)
hc = hc_cluster.fit(df3)

<br/>
Our cluster assignment labels can be viewed via `hc.labels_`:

In [ ]:
hc.labels_

<br/>

Next we will add the cluster assignment labels as a new 'cluster' column in our DataFrame:

In [ ]:
df3['cluster'] = pd.Series(hc.labels_)
df3['cluster'] = df3.cluster.map(pd.Series(['cluster1','cluster2','cluster3']))
df3.head()

<br/>

As before, we will generate a *pairplot()*, coloring each data point based on its cluster assignment:

In [ ]:
sns.pairplot(data=df3,hue='cluster',palette="Set2")
plt.show()

Compare this *pairplot()* to the previous *pairplot()* from the *k-means* results.

Do these results look better or worse than those from the *k-means* exercise?

<br/>
<br/>

Lastly, let's generate another frequency table comparing our known species labels to the *hierarchical clustering* labels:

In [ ]:
ct1 = pd.crosstab(df['species'],df3['cluster'])
sns.heatmap(ct1,annot=True,cbar=False,cmap="Blues")

Looking at the above table, the *setosa* data points are nearly perfectly assigned to one cluster (49/50).

Compared to the results from the *k-means* analysis, the *hierarchical clustering* algorithm did not cluster the *versicolor* and *virginica* data points quite as accurately.

<br/>

---

In machine learning, keep in mind the notion of the [no free lunch theorem](https://en.wikipedia.org/wiki/No_free_lunch_theorem).

Essentially - no single ML algorithm will be optimal for every data set. 

Although *k-means* appears to outperform *hierarchical clustering* for the iris dataset, the opposite could be true on an unrelated dataset.

So - learn about the various unsupervised learning algorithms, their parameters, and be sure to experiment!

---

<br/>
<br/>

For an interesting comparison of clustering algorithms, see: https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py

<br/>
<br/>

<br/>
<br/>
<br/>
<br/>
Created by Scott Perry (perrysc@amazon.com) 20190324